In [1]:
from keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen=ImageDataGenerator(rescale=1/.255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1/.255)

In [4]:
x_train=train_datagen.flow_from_directory("C:/Users/shara/OneDrive/Desktop/pc/ibm project/Dataset/body/training",target_size=(224,224),batch_size=10,class_mode="categorical")
x_test=test_datagen.flow_from_directory("C:/Users/shara/OneDrive/Desktop/pc/ibm project/Dataset/body/validation",target_size=(224,224),batch_size=10,class_mode="categorical")

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [3]:
x_train=train_datagen.flow_from_directory("C:/Users/shara/OneDrive/Desktop/pc/ibm project/Dataset/level/training",target_size=(224,224),batch_size=10,class_mode="categorical")
x_test=test_datagen.flow_from_directory("C:/Users/shara/OneDrive/Desktop/pc/ibm project/Dataset/level/validation",target_size=(224,224),batch_size=10,class_mode="categorical")

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [4]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt


In [5]:
imageSize=[224,224]

In [6]:
vgg= VGG16(input_shape=imageSize+[3],weights="imagenet",include_top=False)

In [7]:
for layer in vgg.layers:
    layer.trainable=False
    


In [8]:
x=Flatten()(vgg.output)

In [9]:
prediction =Dense(3,activation="softmax")(x)

In [10]:
model=Model(inputs=vgg.input,outputs=prediction)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['acc'])

In [13]:
import sys
r=model.fit_generator(x_train,validation_data=x_test,epochs=10,steps_per_epoch=979//10,validation_steps=171//10)

C:\Users\shara\AppData\Local\Temp\ipykernel_9876\901438359.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r=model.fit_generator(x_train,validation_data=x_test,epochs=10,steps_per_epoch=979//10,validation_steps=171//10)


Epoch 1/10
97/97 [==============================] - 385s 4s/step - loss: 78.9176 - acc: 0.5408 - val_loss: 79.5387 - val_acc: 0.5765
Epoch 2/10
97/97 [==============================] - 167s 2s/step - loss: 55.4986 - acc: 0.7038 - val_loss: 95.6995 - val_acc: 0.6059
Epoch 3/10
97/97 [==============================] - 150s 2s/step - loss: 43.6995 - acc: 0.7792 - val_loss: 92.7793 - val_acc: 0.6471
Epoch 4/10
97/97 [==============================] - 191s 2s/step - loss: 28.5603 - acc: 0.8369 - val_loss: 130.3971 - val_acc: 0.5941
Epoch 5/10
97/97 [==============================] - 395s 4s/step - loss: 32.7134 - acc: 0.8442 - val_loss: 127.8051 - val_acc: 0.6059
Epoch 6/10
97/97 [==============================] - 192s 2s/step - loss: 23.7323 - acc: 0.8617 - val_loss: 141.9031 - val_acc: 0.6235
Epoch 7/10
97/97 [==============================] - 148s 2s/step - loss: 20.5681 - acc: 0.8875 - val_loss: 129.6707 - val_acc: 0.6529
Epoch 8/10
97/97 [==============================] - 149s 2s/step 

In [14]:
model.save('level.h5')

In [1]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize
import numpy as np

In [2]:
model1=load_model('body.h5')
model2=load_model('level.h5')

In [9]:
def detect1(frame):
    img=cv2.resize(frame,(224,224))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    if(np.max(img)>1):
        img=img/255.0
    img=np.array([img])
    prediction=model1(img)
    print(prediction)
    label=["front","rear","side"]
    preds=label[np.argmax(prediction)]
    return preds

In [10]:
def detect2(frame):
    img=cv2.resize(frame,(224,224))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    if(np.max(img)>1):
        img=img/255.0
    img=np.array([img])
    prediction=model2(img)
    label=["minor","moderate","severe"]
    preds=label[np.argmax(prediction)]
    return preds

In [13]:
data="C:/Users/shara/OneDrive/Desktop/pc/ibm project/Dataset/level/training/01-minor/0007.JPEG"
image=cv2.imread(data)
print(detect1(image))
print(detect2(image))

tf.Tensor([[3.3851981e-01 1.9293459e-05 6.6146088e-01]], shape=(1, 3), dtype=float32)
side
minor


In [14]:
import re 
import numpy as np
import os
from flask import Flask, app,request,render_template
from tensorflow.keras import models
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.python.ops.gen_array_ops import concat
from tensorflow.keras.applications.inception_v3 import preprocess_input
import requests
from flask import Flask,request,render_template,redirect,url_for
from cloudant.client import Cloudant

In [ ]:
client = Cloudant.iam("48b309ee-528e-4ac3-bb4d-adcf2b863882-bluemix","_i0fpz1gN-ngDO9UG-02mhz6d3MJCuZ-VS4kgkjMziB9",connect=True)

In [ ]:
my_database=client.create_database("my_database")

In [ ]:
model1=load_model("level.h5")
model2=load_model("body.h5")

In [ ]:
app=Flask(__name__)
@app.route('/')
def index():
    return render_template('index.html')

In [ ]:
@app.route('/index.html')
def home():
    return render_template('index.html')

In [ ]:
@app.route('/register')
def register():
    return render_template('register.html')
@app.route('/afterreg',methods=['POST'])
def afterreg():
    x=[x for x in request.form.values()]
    print(x)
    data={
        '_id': x[1],
        'name':x[0],
        'psw':x[2],
    }
    print(data)
    
    query = {'_id':{'$eq': data['_id']}}
    docs = my_database.get_query_result(query)
    print(docs)
    
    print(len(docs.all()))
    if (len(docs.all())==0):
        url = my_database.create_document(data)
        return render_template('register.html',pred="Registration Successful, please login using your details")
    else:
         return render_template('register.html',pred="You are already a member, please login using your details")
        




In [ ]:
@app.route('/Login')
def login():
    return render_template('Login.html')
@app.route('/afterLogin',methods=['POST'])
def afterlogin():
    user = request.form['_id']
    passw=request.form['psw']
    print(user,passw)
    
    query = {'_id':{'$eq': user}}
    docs = my_database.get_query_result(query)
    print(docs)
    
    print(len(docs.all()))
    if (len(docs.all())==0):
        return render_template('Login.html',pred="The username is not found.")
    else:
         if((user==docs[0][0]['_id'] and passw==docs[0][0]['psw'])):
            return redirect(url_for('prediction'))
         else:
            print('Invalid User')
        




In [ ]:
@app.route('/logout')
def logout():
    return render_template('logout.html')

In [ ]:
@app.route('/prediction')
def prediction():
    return render_template('prediction.html')

In [ ]:
@app.route("/results",methods=["GET","POST"])
def res():
    if request.method=="POST":
        f=request.files["image"]
        basepath=os.path.dirname(_file_) 
        filepath=os.path.join(basepath,'upload',f.filename)
        f.save(filepath)
        img=image.load_img(filepath, target_size=(256,256))
        x=image.img_to_array(img)
        x=np.expand_dims(x,axis=0)
        img_data=preprocess_input(x)
        prediction1=np.argmax(model1.predict(img_data)) 
        prediction2=np.argmax(model2.predict(ing_data))
        index1=["front","rear","side"]
        index2=["minor","moderate","severe"]

        result1=index1[prediction1]
        result2=index2[prediction2]
        if (result1 == "front" and result2 == "minor"):
            value = "3000 - 5000 INR"
        elif (result1 == "front" and result2 == "moderate"):
            value = "6000 8000 INR"
        elif (result1 == "front" and result2 == "severe"):
            value = "9000 11000 INR"

        elif (result1 == "rear" and result2 == "minor"):
            value = "4000 - 6000 INR"

        elif (result1 == "rear" and result2 == "moderate"):
            value = "7000 9000 INR"

        elif (result1 == "rear" and result2 == "severe"):
            value = "11000 - 13000 INR"

        elif (result1 == "side" and result2 == "minor"):
            value = "6000 - 8000 INR"

        elif (result1 == "side" and result2 == "moderate"):
            value = "9000 - 11000 INR"

        elif (result1 == "side" and result2 == "severe"):
            value = "12000 - 15000 INR"

        else:
            value = "16000 - 50000 INR"
        return render_template('prediction.html', prediction=value)    
if __name__ == '__main__':
    app.run()
   

In [ ]:
if __name__ == '__main__':
    app.run(debug=False,port=8080)